In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import logging

2025-07-05 11:12:50.296964: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-05 11:12:50.405021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751695970.450009    3977 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751695970.471208    3977 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751695970.555678    3977 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
tf.autograph.set_verbosity(0)
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [3]:
## Synthetic Data
rng = np.random.default_rng(2)
xtr = rng.random(400).reshape(-1,2)
xtr[:,1] = xtr[:,1] * 4 + 11.5         
xtr[:,0] = xtr[:,0] * (285-150) + 150  
ytr = np.zeros(len(xtr))

i=0
for t,d in xtr:
    y = -3/(260-175)*t + 21
    if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
        ytr[i] = 1
    else:
        ytr[i] = 0
    i += 1

ytr = ytr.reshape(-1,1)

In [4]:
print(xtr.shape, ytr.shape)

print(f"Temp Max: {np.max(xtr[:,0]):0.2f}, Min: {np.min(xtr[:,0]):0.2f}")
print(f"Duration Max: {np.max(xtr[:,1]):0.2f}, Min: {np.min(xtr[:,1]):0.2f}")

(200, 2) (200, 1)
Temp Max: 284.99, Min: 151.32
Duration Max: 15.45, Min: 11.51


In [16]:
def normalize(x):
    mu = np.mean(x, axis=0)
    std = np.std(x, axis=0)

    std_adj = np.where(std == 0, 1, std)
    
    x_norm = (x-mu) / std_adj
    return x_norm

In [17]:
x_norm = normalize(xtr)

In [18]:
print(f"Temp Max: {np.max(x_norm[:,0]):0.2f}, Min: {np.min(x_norm[:,0]):0.2f}")
print(f"Duration Max: {np.max(x_norm[:,1]):0.2f}, Min: {np.min(x_norm[:,1]):0.2f}")

Temp Max: 1.66, Min: -1.69
Duration Max: 1.79, Min: -1.70


In [19]:
def sigmoid(z):
    g = 1/(1+np.exp(-z))
    return g

In [40]:
def dense_layer(a_in, W, b, activation=sigmoid):
    units = W.shape[1]
    a_out = np.zeros(units)
    print("a_in shape:", a_in.shape)
    print("W shape:", W.shape)
    for j in range(units):
        w = W[:,j]
        a_out[j] = activation(w.dot(a_in) + b[j])
    return a_out

In [41]:
def sequential(x, W1, b1, W2, b2):
    a1 = dense_layer(x, W1, b1)
    a2 = dense_layer(a1, W2, b2)
    return a2

In [42]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

In [43]:
def predict(x, W1, b1, W2, b2):
    m = x.shape[0]
    p = np.zeros((m,1))
    for i in range(m):
        p[i,0] = sequential(x[i], W1, b1, W2, b2)
    return p

In [44]:
X_tst = np.array([
    [200,13.9],
    [200,17]]) 

In [45]:
X_tstn = normalize(X_tst)
print(X_tstn)

[[ 0. -1.]
 [ 0.  1.]]


In [46]:
predictions = predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp)

a_in shape: (2,)
W shape: (2, 3)
a_in shape: (3,)
W shape: (3, 1)
a_in shape: (2,)
W shape: (2, 3)
a_in shape: (3,)
W shape: (3, 1)


/tmp/ipykernel_3977/1830987532.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p[i,0] = sequential(x[i], W1, b1, W2, b2)


In [48]:
yhat = (predictions >= 0.5).astype(int)
yhat

array([[1],
       [0]])